# Merge 2011 and 2019 datasets of Italian dwellings by occupancy status

## Import datasets and create dataframes

In [1]:
import pandas as pd

In [2]:
df_2011 = pd.read_csv('data/dwellings by occupancy status 2011.csv')
df_2011.sample(3)

,Territory,occupied conventional dwellings by at least one resident person,occupied conventional dwellings by not resident people,unoccupied conventional dwellings,all items
87,Asti,94279,45,30241,124565
51,Varese,361978,165,57288,419431
74,Verbano-Cusio-Ossola,71729,26,39210,110965


In [3]:
df_2019 = pd.read_csv('data/dwellings by occupancy status 2019.csv')
df_2019.sample(3)

,Territory,occupied conventional dwellings,unoccupied conventional dwellings,conventional dwellings
104,Cagliari,188050,31269,219319
66,Roma,1892483,377083,2269566
54,Siena,116021,47958,163979


## Compare lengths and check duplicates

In [4]:
len(df_2011)

110

In [5]:
len(df_2019)

107

In [6]:
df_2011[df_2011['Territory'].duplicated()]

,Territory,occupied conventional dwellings by at least one resident person,occupied conventional dwellings by not resident people,unoccupied conventional dwellings,all items


In [7]:
df_2019[df_2019['Territory'].duplicated()]

,Territory,occupied conventional dwellings,unoccupied conventional dwellings,conventional dwellings


2011 dataset has 3 more rows than 2019: it's not because of duplicates, but because some provinces have changed

## Adapt structure of 2011 data to the one of 2019 data

In [8]:
df_2011['occupied convetional dwellings'] = df_2011['occupied conventional dwellings by at least one resident person']+df_2011['occupied conventional dwellings by not resident people']
df_2011.sample(3)

,Territory,occupied conventional dwellings by at least one resident person,occupied conventional dwellings by not resident people,unoccupied conventional dwellings,all items,occupied convetional dwellings
1,Cosenza,283897,861,198166,482924,284758
46,Ferrara,157453,1259,59797,218509,158712
9,Bergamo,437474,128,128768,566370,437602


In [9]:
df_2011 = df_2011.drop(columns=['occupied conventional dwellings by at least one resident person', 'occupied conventional dwellings by not resident people'])
df_2011.sample(3)

,Territory,unoccupied conventional dwellings,all items,occupied convetional dwellings
98,Mantova,24496,188049,163553
55,Grosseto,53361,151005,97644
79,Massa-Carrara,35708,120462,84754


In [10]:
df_2011 = df_2011.rename(columns={'all items': 'conventional dwellings'})
df_2011.sample(3)

,Territory,unoccupied conventional dwellings,conventional dwellings,occupied convetional dwellings
80,Benevento,35155,143610,108455
108,Lodi,10407,102545,92138
22,Caserta,88609,407899,319290


## Add label for year in columns

In [11]:
for col in df_2011.columns:
    if col != "Territory":
        df_2011 = df_2011.rename(columns={col: "2011 "+col})
df_2011.sample(3)

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
37,Frosinone,68658,258974,190316
87,Asti,30241,124565,94324
22,Caserta,88609,407899,319290


In [12]:
for col in df_2019.columns:
    if col != "Territory":
        df_2019 = df_2019.rename(columns={col: "2019 "+col})
df_2019.sample(3)

,Territory,2019 occupied conventional dwellings,2019 unoccupied conventional dwellings,2019 conventional dwellings
73,Campobasso,92176,76893,169069
38,Piacenza,128281,59876,188157
36,Trieste,115575,29428,145003


## Merge

In [13]:
df_merged = pd.merge(df_2011, df_2019, left_on="Territory", right_on="Territory", how="inner") 

In [14]:
df_merged.sample(3)

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings,2019 occupied conventional dwellings,2019 unoccupied conventional dwellings,2019 conventional dwellings
91,Oristano,26537,91572,65035,66870,38828,105698
88,Novara,28397,184592,156195,159604,60204,219808
63,Padova,45551,410443,364892,386498,81410,467908


In [15]:
len(df_merged)

105

Some rows are missed in the inner merging

In [16]:
df_outer_merged= pd.merge(df_2011, df_2019, left_on="Territory", right_on="Territory", how="outer", indicator=True)
df_outer_merged.sample(3)

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings,2019 occupied conventional dwellings,2019 unoccupied conventional dwellings,2019 conventional dwellings,_merge
97,Terni,25096.0,121935.0,96839.0,99656.0,38466.0,138122.0,both
48,Aosta,58731.0,117293.0,58562.0,NaN,NaN,NaN,left_only
74,Verbano-Cusio-Ossola,39210.0,110965.0,71755.0,72188.0,72433.0,144621.0,both


In [17]:
unmerged = df_outer_merged[df_outer_merged["_merge"] != "both"]
unmerged

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings,2019 occupied conventional dwellings,2019 unoccupied conventional dwellings,2019 conventional dwellings,_merge
35,Olbia-Tempio,71614.0,135418.0,63804.0,NaN,NaN,NaN,left_only
48,Aosta,58731.0,117293.0,58562.0,NaN,NaN,NaN,left_only
103,Carbonia-Iglesias,18913.0,70947.0,52034.0,NaN,NaN,NaN,left_only
105,Ogliastra,14652.0,38973.0,24321.0,NaN,NaN,NaN,left_only
109,Medio Campidano,9971.0,48919.0,38948.0,NaN,NaN,NaN,left_only
110,Valle d'Aosta / Vallée d'Aoste,NaN,NaN,NaN,59736.0,78317.0,138053.0,right_only
111,Sud Sardegna,NaN,NaN,NaN,148036.0,74562.0,222598.0,right_only


### Correct mismathching rows

#### *Aosta* and *Valle d'Aosta / Vallée d'Aoste* need to be renamed *Valle d'Aosta*

In [18]:
df_2011[df_2011["Territory"]=="Aosta"]

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
48,Aosta,58731,117293,58562


In [19]:
df_2011.at[48, "Territory"]="Valle d'Aosta"
df_2011.loc[48,:]

Territory                                 Valle d'Aosta
2011 unoccupied conventional dwellings            58731
2011 conventional dwellings                      117293
2011 occupied convetional dwellings               58562
Name: 48, dtype: object

In [20]:
df_2019[df_2019["Territory"]=="Valle d'Aosta / Vallée d'Aoste"]

,Territory,2019 occupied conventional dwellings,2019 unoccupied conventional dwellings,2019 conventional dwellings
8,Valle d'Aosta / Vallée d'Aoste,59736,78317,138053


In [21]:
df_2019.at[8, "Territory"]="Valle d'Aosta"
df_2019.loc[8,:]

Territory                                 Valle d'Aosta
2019 occupied conventional dwellings              59736
2019 unoccupied conventional dwellings            78317
2019 conventional dwellings                      138053
Name: 8, dtype: object

#### *Carbonia-Iglesias* and *Medio Campidano* need to be joined in *Sud Sardegna*

In [22]:
df_2011[df_2011["Territory"]=="Medio Campidano"]

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
109,Medio Campidano,9971,48919,38948


In [23]:
df_2011[df_2011["Territory"]=="Carbonia-Iglesias"]

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
103,Carbonia-Iglesias,18913,70947,52034


In [24]:
old_territories_to_join = pd.concat([df_2011[df_2011["Territory"]=="Carbonia-Iglesias"], df_2011[df_2011["Territory"]=="Medio Campidano"]])
old_territories_to_join

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
103,Carbonia-Iglesias,18913,70947,52034
109,Medio Campidano,9971,48919,38948


In [25]:
Sud_Sardegna_row = old_territories_to_join.sum().to_frame().transpose()
Sud_Sardegna_row

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
0,Carbonia-IglesiasMedio Campidano,28884,119866,90982


In [26]:
Sud_Sardegna_row.at[0, "Territory"]="Sud Sardegna"
Sud_Sardegna_row

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
0,Sud Sardegna,28884,119866,90982


In [27]:
df_2011 = pd.concat([df_2011, Sud_Sardegna_row], ignore_index=True)
df_2011.tail(2)

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
109,Medio Campidano,9971,48919,38948
110,Sud Sardegna,28884,119866,90982


In [28]:
df_2011=df_2011.drop([103, 109])

#### *Nuoro* has to include also the sum of the values of *Ogliastra* and *Olbia-Tempio*

In [29]:
df_2011[df_2011["Territory"]=="Nuoro"]

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
96,Nuoro,25507,89592,64085


In [30]:
df_2011[df_2011["Territory"]=="Olbia-Tempio"]

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
35,Olbia-Tempio,71614,135418,63804


In [31]:
df_2011[df_2011["Territory"]=="Ogliastra"]

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
105,Ogliastra,14652,38973,24321


In [32]:
old_territories_to_merge = pd.concat([df_2011[df_2011["Territory"]=="Ogliastra"], df_2011[df_2011["Territory"]=="Olbia-Tempio"], df_2011[df_2011["Territory"]=="Nuoro"]])
old_territories_to_merge

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
105,Ogliastra,14652,38973,24321
35,Olbia-Tempio,71614,135418,63804
96,Nuoro,25507,89592,64085


In [33]:
Nuoro_row = old_territories_to_merge.sum().to_frame().transpose()
Nuoro_row

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
0,OgliastraOlbia-TempioNuoro,111773,263983,152210


In [34]:
df_2011=df_2011.drop([35, 96, 105])

In [35]:
Nuoro_row.at[0, "Territory"]="Nuoro"
Nuoro_row

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
0,Nuoro,111773,263983,152210


In [36]:
df_2011 = pd.concat([df_2011, Nuoro_row], ignore_index=True)
df_2011.tail(2)

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings
105,Sud Sardegna,28884,119866,90982
106,Nuoro,111773,263983,152210


## Datasets can now be merged

In [37]:
len(df_2011)==len(df_2019) && len(df_2011)==107

True

In [38]:
df_outer_merged= pd.merge(df_2011, df_2019, left_on="Territory", right_on="Territory", how="outer", indicator=True)
unmerged = df_outer_merged[df_outer_merged["_merge"] != "both"]
unmerged

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings,2019 occupied conventional dwellings,2019 unoccupied conventional dwellings,2019 conventional dwellings,_merge


In [39]:
df_merged = pd.merge(df_2011, df_2019, left_on="Territory", right_on="Territory", how="inner") 
df_merged.sample(3)

,Territory,2011 unoccupied conventional dwellings,2011 conventional dwellings,2011 occupied convetional dwellings,2019 occupied conventional dwellings,2019 unoccupied conventional dwellings,2019 conventional dwellings
64,Padova,45551,410443,364892,386498,81410,467908
15,Salerno,108124,504320,396196,419213,201994,621207
77,Arezzo,36044,174615,138571,142824,52937,195761


In [40]:
df_merged.to_csv('data/2011-2019 dwellings by occupancy status.csv', index=False)